In [25]:
import numpy as np
import matplotlib.pyplot as plt
import PIL

def brightness(img, beta):
  new_img = np.clip((img / 255) + (beta / 255), 0, 1)
  return new_img

def contrast(img, alpha):
  new_img = np.clip(img * alpha / 255, 0, 1)
  return new_img

def flip(img, direction):
  new_img = np.flip(img, axis=direction)
  return new_img

def grayscale(img):
  luma_scalars = np.array([0.3, 0.59, 0.11])
  new_img = np.dot(img / 255, luma_scalars)
  return new_img

def sepia(img):
  new_rgb_scalars = np.array([[0.393, 0.349, 0.272], [0.769, 0.686, 0.534], [0.189, 0.168, 0.131]])
  new_img = np.clip(np.matmul(img / 255, new_rgb_scalars), 0, 1)
  return new_img

def padding(img, size):
  height, width, channels = img.shape
  img_padding = np.zeros((height + 2 * size, width + 2 * size, channels))
  img_padding[size : height + size, size : width + size] = img
  return img_padding

def blur(img, k):
  img_padding = padding(img, k // 2)
  row, col = img_padding.shape[:2]
  new_img = []
  for i in range(row - k + 1):
    for j in range(col - k + 1):
      new_img.append(np.sum(img_padding[i:i+k, j:j+k], axis=(0, 1)) / (k * k * 1.0))
  new_img = np.array(new_img, dtype=np.uint8)
  new_img = np.reshape(new_img, img.shape)
  return new_img

def sharpen(img):
  kernel = np.array([[0, -1, 0],
                   [-1,  5, -1],
                   [0, -1, 0]])
  k = 3
  img_padding = padding(img, k // 2)
  row, col = img_padding.shape[:2]
  new_img = []
  for i in range(row - k + 1):
    for j in range(col - k + 1):
      new_img.append(
          np.clip(np.sum(
          (np.reshape(img_padding[i:i+k, j:j+k], (k * k, img_padding.shape[2]))
            * np.reshape(kernel, (k * k, 1))), axis=0), 0, 255))
  new_img = np.array(new_img, dtype=np.uint8)
  new_img = np.reshape(new_img, img.shape)
  return new_img

def center_crop(img, crop_height, crop_width):
  height, width = img.shape[:2]
  center = [height // 2, width // 2]
  new_img = img[center[0] - (crop_height//2):center[0] + (crop_height // 2), center[1] - (crop_width//2):center[1] + (crop_width // 2)]
  return new_img

def generate_circle_array(array_size):
  circle_array = np.zeros((array_size, array_size), dtype=np.uint8)

  center = array_size // 2
  radius = center

  y, x = np.ogrid[-center:array_size-center, -center:array_size-center]
  mask = x**2 + y**2 <= radius**2
  circle_array[mask] = 1

  return circle_array

def circle_drop(img):
  size = img.shape[0] if img.shape[0] <= img.shape[1] else img.shape[1]
  circle_array = generate_circle_array(img.shape[0])
  circle_mask = circle_array.astype(bool)
  red_channel, green_channel, blue_channel = img[:, :, 0], img[:, :, 1], img[:, :, 2]

  crop_red_channel = np.zeros_like(red_channel)
  crop_red_channel[circle_mask] = red_channel[circle_mask]

  crop_green_channel = np.zeros_like(green_channel)
  crop_green_channel[circle_mask] = green_channel[circle_mask]

  crop_blue_channel = np.zeros_like(blue_channel)
  crop_blue_channel[circle_mask] = blue_channel[circle_mask]

  new_img = np.stack([crop_red_channel, crop_green_channel, crop_blue_channel], axis=-1)
  return new_img

def generate_ellipse_array(array_size):
  size = int(np.ceil(array_size * 5/4))
  ellipse_array = np.zeros((size, size), dtype=np.uint8)

  center = size // 2
  radius = center

  a = (size / 2)**2
  b = int(np.ceil(array_size * np.sqrt(2) / 4))**2

  y, x = np.ogrid[-center:size-center, -center:size-center]
  mask = (x**2/a) + (y**2/b) <= 1
  ellipse_array[mask] = 255

  height, width = ellipse_array.shape
  diagonal_length = int(np.ceil(np.sqrt(height**2 + width**2)))

  rotated_arr = np.zeros((height, width), dtype=ellipse_array.dtype)

  center_x, center_y = width // 2, height // 2
  cos45, sin45 = np.cos(np.pi/4), np.sin(np.pi/4)

  for y in range(height):
      for x in range(width):
          x_rot = int((x - center_x) * cos45 + (y - center_y) * sin45 + center_x)
          y_rot = int((y - center_y) * cos45 - (x - center_x) * sin45 + center_y)
          if 0 <= x_rot < width and 0 <= y_rot < height:
              rotated_arr[y_rot, x_rot] = ellipse_array[y, x]

  return center_crop(rotated_arr, array_size, array_size)

def ellipse_drop(img):
  size = img.shape[0] if img.shape[0] <= img.shape[1] else img.shape[1]

  ellipse_array = generate_ellipse_array(size) + flip(generate_ellipse_array(size), 0)

  ellipse_mask = ellipse_array.astype(bool)
  red_channel, green_channel, blue_channel = img[:, :, 0], img[:, :, 1], img[:, :, 2]

  crop_red_channel = np.zeros_like(red_channel)
  crop_red_channel[ellipse_mask] = red_channel[ellipse_mask]

  crop_green_channel = np.zeros_like(green_channel)
  crop_green_channel[ellipse_mask] = green_channel[ellipse_mask]

  crop_blue_channel = np.zeros_like(blue_channel)
  crop_blue_channel[ellipse_mask] = blue_channel[ellipse_mask]

  new_img = np.stack([crop_red_channel, crop_green_channel, crop_blue_channel], axis=-1)
  return new_img

def input_process(img, f, img_name):
  if f == 1:
    print("[BRIGHTNESS] Adjust brightness:")
    beta = int(input())
    new_img_name = img_name + "_brightness"
    img_brightness = brightness(img, beta)
    res = PIL.Image.fromarray((img_brightness * 255).astype(np.uint8))
    res.save(f"./{new_img_name}.png")
  elif f == 2:
    print("[CONTRAST] Adjust contrast (> 0):")
    alpha = float(input())
    if alpha > 0:
      new_img_name = img_name + "_contrast"
      img_contrast = contrast(img, alpha)
      res = PIL.Image.fromarray((img_contrast * 255).astype(np.uint8))
      res.save(f"./{new_img_name}.png")
    else:
      print("[CONTRAST] Invalid input")
  elif f == 3:
    print("[FLIP] Flip image (0: vertical, 1: horizontal):")
    direction = int(input())
    if direction in range(0, 2):
      new_img_name = img_name + "_flip"
      img_flip = flip(img, direction)
      res = PIL.Image.fromarray(img_flip)
      res.save(f"./{new_img_name}.png")
    else:
      print("[FLIP] Invalid input")
  elif f == 4:
    img_grayscale = grayscale(img)
    new_img_name = img_name + "_grayscale"
    res = PIL.Image.fromarray((img_grayscale * 255).astype(np.uint8))
    res.save(f"./{new_img_name}.png")
    
    img_sepia = sepia(img)
    new_img_name = img_name + "_sepia"
    res = PIL.Image.fromarray((img_sepia * 255).astype(np.uint8))
    res.save(f"./{new_img_name}.png")
  elif f == 5:
    print("[BLUR] Adjust blur (k = {3, 5, 7,...}):")
    kernel_size = int(input())
    img_blur = blur(img, kernel_size)
    new_img_name = img_name + "_blur"
    res = PIL.Image.fromarray(img_blur)
    res.save(f"./{new_img_name}.png")

    img_sharpen = sharpen(img)
    new_img_name = img_name + "_sharpen"
    res = PIL.Image.fromarray(img_sharpen)
    res.save(f"./{new_img_name}.png")
  elif f == 6:
    height, width = img.shape[:2]
    print("[CENTER DROP] Crop size (height <= ", height, ", width <= ", width, ")")
    print("Crop height: ")
    crop_height = int(input())
    print("Crop width: ")
    crop_width = int(input())
    if crop_height <= height and crop_width <= width:
      img_center_crop = center_crop(img, crop_height, crop_width)
      new_img_name = img_name + "_center_drop"
      res = PIL.Image.fromarray(img_center_crop)
      res.save(f"./{new_img_name}.png")
    else:
      print("[CROP] invalid input")
  elif f == 7:
    img_circle_drop = circle_drop(img)
    new_img_name = img_name + "_circle_drop"
    res = PIL.Image.fromarray(img_circle_drop)
    res.save(f"./{new_img_name}.png")
    
    img_ellipse_drop = ellipse_drop(img)
    new_img_name = img_name + "_ellipse_drop"
    res = PIL.Image.fromarray(img_ellipse_drop)
    res.save(f"./{new_img_name}.png")

def main():
  print("Image path:")
  img_path = input()
  img_name = img_path.split('/')[-1]
  img_name = img_name.split('.')[0]
  img = np.array(PIL.Image.open(img_path))

  print("Function (0-7): ")
  f = int(input())

  if f == 0:
    for x in range (8):
      input_process(img, x + 1, img_name)
  elif f in range (1, 8):
    input_process(img, f, img_name)
  else:
    print("invalid input")
    

main()

Image path:
Function (0-7): 
[FLIP] Flip image (0: vertical, 1: horizontal):
